In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor
from torch.optim import Adam
import os
import pandas as pd
import albumentations as A
import numpy as np
from PIL import Image

In [ ]:
batch_size = 30
num_epochs = 1200

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class TicTacToeDataset(Dataset):
    def __init__(self, img_dir, df, transform=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(df)
        self.n = len(os.listdir(self.img_dir))
        self.transform = transform

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        idx, img_path, label = self.df.iloc[idx, :].values
        label = label+1
        image = Image.open(os.path.join(self.img_dir, img_path))
        image = np.array(image)
        if self.transform:
            image = self.transform(image=image)['image']
        image = to_tensor(image)
        return image, label

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            self._block(1, 32, 3),
            self._block(32, 48, 3),
            self._block(48, 64, 3),
            self._block(64, 80, 3),
            self._block(80, 96, 3),
            self._block(96, 112, 3),
            self._block(112, 128, 3),
            self._block(128, 144, 3),
            self._block(144, 160, 3),
            self._block(160, 176, 3),
            Flatten(),
            nn.Linear(11264, 3, bias=False),
            nn.BatchNorm1d(3),
        )

    def _block(self, input_dim, output_dim, kernel_size):
        return nn.Sequential(
            nn.Conv2d(input_dim, output_dim, kernel_size, bias=False),
            nn.BatchNorm2d(output_dim),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.seq(x)
        return F.log_softmax(x, dim=1)

class Flatten(nn.Module):
    def forward(self, x):
        return torch.flatten(x.permute(0, 2, 3, 1), 1)


In [ ]:
model = Model().to(device)
optimizer = Adam(model.parameters(), lr=0.001)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomCrop(26, 26, p=0.4),
    A.InvertImg(p=0.25),
    A.GaussNoise(p=0.35),
    A.RandomBrightnessContrast(brightness_limit=(-0.15, 0.15), contrast_limit=(-0.15, 0.15), p=0.5),
    A.Resize(28, 28, p=1.0, interpolation=Image.NEAREST),
])

class_sample_count = [800, 1606]
weights = 1 / torch.Tensor(class_sample_count).float()
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, batch_size)

dataset = TicTacToeDataset('drive/MyDrive/fields', 'drive/MyDrive/fields.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, sampler=sampler)

In [ ]:
def main():
    for epoch in range(num_epochs):
        print(epoch)
        for (value, label) in dataloader:
            value, label = value.to(device), label.to(device)
            output = model(value)
            loss = F.nll_loss(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [ ]:
main()

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/tictactoeField.pth')